## Preparation of DICOM Images and Labels for Neural Network Training with Torch

### Stacking 2D Images into 3D and Padding

In [ ]:
import torch
import os
import pydicom
import numpy as np
from monai.data import Dataset
from monai.transforms import LoadImage

class MedicalImageDataset(Dataset):
    def __init__(self, image_dir, label_dir, transforms=None):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.transforms = transforms
        # the self.patients list is filled with the names of directories in image_dir when then directory name includes "PANCREAS"
        self.patients = [f for f in sorted(os.listdir(image_dir)) if "PANCREAS" in f]
        self.max_depth = self.calculate_max_depth()

    def calculate_max_depth(self):
        max_depth = 0
        for patient_folder in self.patients:
            patient_image_dir = os.path.join(self.image_dir, patient_folder)
            volume = self.load_dicom_volume(patient_image_dir) # finds the number of slices in the folder
            if volume.shape[0] > max_depth:
                max_depth = volume.shape[0]
        return max_depth

    def load_dicom_volume(self, patient_image_dir):
        subfolder = next(os.walk(patient_image_dir))[1][0]
        deepest_folder = next(os.walk(os.path.join(patient_image_dir, subfolder)))[1][0]
        final_path = os.path.join(patient_image_dir, subfolder, deepest_folder)
        files = sorted(os.listdir(final_path), key=lambda x: pydicom.dcmread(os.path.join(final_path, x)).InstanceNumber)
        # os.listdir(final_path): refers to the folder with the dicom files
        # sorted() usually sorts them in alphabetical, but using the key parameter makes sure a specific order is used
        # key=lambda x: pydicom.dcmread(os.path.join(final_path, x)).InstanceNumber:
            # lambda x: defines an anonymous function where x is the name of the variable that represents each element in the list
            # pydicom.dcmread(os.path.join(final_path, x)).InstanceNumber : the instance number of each dicom image is used to ensure that the images are sorted in consecutive, sequential order.

        # stacks all of the 2D slices for each patient into a 3D array
        volume = np.stack([pydicom.dcmread(os.path.join(final_path, f)).pixel_array for f in files])

        # converts the numpy array into a torch tensor
        return torch.from_numpy(volume).float()  # No need to add batch dimension manually

    def __len__(self):
        return len(self.patients)

    def __getitem__(self, idx):
        patient_folder = self.patients[idx]
        patient_image_dir = os.path.join(self.image_dir, patient_folder)
        patient_label_path = os.path.join(self.label_dir, f"label{patient_folder.split('_')[-1]}.nii.gz")
        
        volume = self.load_dicom_volume(patient_image_dir)

        # label image is loaded with MONAI's LoadImage where only the image is loaded with image_only = True
        label = LoadImage(image_only=True)(patient_label_path)
        # rearranges the axes of the label array to match the conventional layout expected by PyTorch models (num_channels x height x width)
        label = np.transpose(label, (2, 0, 1))
        # turns numpy array into torch tensor
        label = torch.from_numpy(label).float()

        # Padding to maximum depth for depth (slice axis)
        pad_size = self.max_depth - volume.shape[0]
        volume = torch.nn.functional.pad(volume, (0, 0, 0, 0, 0, pad_size))
        label = torch.nn.functional.pad(label, (0, 0, 0, 0, 0, pad_size))

        # Correcting shape to (512, 512, padded_num_slices, 1)
        volume = volume.permute(2, 1, 0).unsqueeze(-1)  # changes the shape of the volume tensor to fit the format expected by neural networks of (H, W, D, C)
        # permute function allows for changing dimensions of the tensor
        # unsqueeze function adds a new dimension at the specified index. Here the index is -1 which indicates that the new dimension added into the last position. 
        label = label.permute(2, 1, 0).unsqueeze(-1)    # changes the shape of the volume tensor to fit the format expected by neural networks of (H, W, D, C)

        if self.transforms:
            volume = self.transforms(volume)
            label = self.transforms(label)

        return volume.squeeze(0), label.squeeze(0)  # Ensure removing any singleton dimension at batch axis

### Setting up and testing DataLoader

In [ ]:
import os
import numpy as np
import pydicom
import torch
from torch.utils.data import DataLoader, Dataset
from monai.transforms import Compose, ScaleIntensity, EnsureType
from monai.transforms import LoadImage

# Transforms and DataLoader
transforms = Compose([
    ScaleIntensity(), # normalizes or scales image intensities
    EnsureType(dtype=torch.float32) # ensures that the data type of the tensors after transformationis torch.float32 which is standard for PyTorch models
])

# Paths
image_root_dir = "/Users/asmit/Programming/SDS323/Final_Project/manifest-1599750808610/Pancreas-CT"
label_root_dir = "/Users/asmit/Programming/SDS323/Final_Project/TCIA_pancreas_labels-02-05-2017"

dataset = MedicalImageDataset(image_root_dir, label_root_dir, transforms=transforms)

# DataLoader is initialized which is standard for PyTorch usage
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
# note that the dataset has volume, label at this point
# batch_size = 1 ensures that each item retried by the DataLoader will contain one sample from the dataset which in this case is one volume and one label

# Test DataLoader
for volume, label in dataloader:
    # Squeeze out the batching dimension which is the first dimension
    volume, label = volume.squeeze(0), label.squeeze(0) # Comment if permute block is commented above
    print("Volume shape:", volume.shape, "Label shape:", label.shape)
    break # Checking only the first batch

### EDA: Exploratory Data Analysis
##### Initial General Notes: There is an imbalance of classes in the data as there are multiple organs in the image of the abdomen other than the pancreas. There are no missing data as the image to label ratio is clearly 1:1.
#### Pixel Intensity

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_intensity_distribution(dataloader, num_batches=1, samples_per_volume=100000):
    volume_intensities = []
    label_intensities = []

    for i, (volume, label) in enumerate(dataloader):

        # Stopping criteria
        if i >= num_batches:
            break

        # Squeeze out channel and batch singleton dimensions
        volume = volume.squeeze().numpy()
        label = label.squeeze().numpy()

        # Iterate through each slice in batch
        for vol_slice, lbl_slice in zip(volume, label):

            # Filter out padding values
            non_padded_pixels = vol_slice[vol_slice > 0]  # Adjust this threshold as needed
            labeled_pixels = vol_slice[lbl_slice > 0]

            # Randomly sample pixels to reduce the data size
            if len(non_padded_pixels) > samples_per_volume:
                sampled_volume_pixels = np.random.choice(non_padded_pixels, samples_per_volume, replace=False)
            else:
                sampled_volume_pixels = non_padded_pixels
            
            if len(labeled_pixels) > samples_per_volume:
                sampled_label_pixels = np.random.choice(labeled_pixels, samples_per_volume, replace=False)
            else:
                sampled_label_pixels = labeled_pixels

            volume_intensities.extend(sampled_volume_pixels)
            label_intensities.extend(sampled_label_pixels)
    
    # Plot the histogram of the sampled intensities
    fig, axs = plt.subplots(1, 2, figsize=(15, 6))
    
    axs[0].hist(volume_intensities, bins=256, color='skyblue', alpha=0.75)  # Adjust bins if needed
    axs[0].set_title('Pixel Intensity Distribution of Volumes')
    axs[0].set_xlabel('Pixel Intensity')
    axs[0].set_ylabel('Frequency')
    axs[0].grid(True)

    axs[1].hist(label_intensities, bins=30, color='lightcoral', alpha=0.75)  # Adjust bins if needed
    axs[1].set_title('Pixel Intensity Distribution of Labeled Pancreas')
    axs[1].set_xlabel('Pixel Intensity')
    axs[1].set_ylabel('Frequency')
    axs[1].grid(True)

    plt.tight_layout()
    plt.show()

#### Sample Splitting

In [ ]:
# Remove the aggregate dataloader from memory
del dataloader

In [ ]:
import torch
from torch.utils.data import DataLoader, random_split

def create_data_loaders(dataset, batch_size=1):

    # Determine sizes of each split
    total_size = len(dataset)
    train_size = int(0.6 * total_size)
    val_size = int(0.2 * total_size)
    test_size = total_size - train_size - val_size

    # Randomly split the dataset into train, cv, and test sets
    train_dataset, val_dataset, test_dataset = random_split(
        dataset, [train_size, val_size, test_size],
        generator=torch.Generator().manual_seed(832) # Set seed
    )

    # Create data loaders for each split
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader

# Create dataloaders
dataset = MedicalImageDataset(image_root_dir, label_root_dir, transforms=transforms)
train_loader, val_loader, test_loader = create_data_loaders(dataset, batch_size=1)